# Python 101
## XII. Command line interfaces

---

## I. `sys.argv`
Python's built-in low level solution is `sys.argv`. It gives full control over the script arguments but building complex comand line interface is cumbersome.

In [ ]:
import os
import sys

import subprocess

In [ ]:
def run(content, commands):
    with open('test.py', 'w') as f:
        f.write(content)

    for command in commands:
        print(f'Command {command} output:')
        result = subprocess.run(command.split(), capture_output=True)
        print(f"STDOUT: {result.stdout.decode('latin1')}\n"
              f"STDERR: {result.stderr.decode('latin1')}")
        print('-' * 80)

    os.remove('test.py')

In [ ]:
content = """
import sys

if __name__ == '__main__':
    fn, *args = sys.argv
    print(f'Script {fn} has the following arguments:')
    for arg in args:
        print(f'- {arg}')
"""

run(content, ['python test.py arg1 arg2 arg3'])

In [ ]:
content = """
import sys

if __name__ == '__main__':
    fn, *args = sys.argv
    counting, listing, isin, nonkw = False, False, None, []
    for i, arg in enumerate(args):
        if arg in ('-h', '--help'):
            print(f'{fn} script\\'s help.')
            print(f'usage: test.py [-h|--help] [-c|--count] [-l|--list] [-i|--isin VAL] ARG [ARG ...]')
        elif arg in ('-c', '--count'):
            counting = True
        elif arg in ('-l', '--list'):
            listing = True
        elif arg in ('-i', '--isin'):
            if len(args) > i + 1:
                isin = i + 1
            else:
                print('Missing keyword argument!')
                sys.exit(1)
        elif i != isin:
            nonkw.append(arg)
    if counting:
        print(f'Number of arguments: {len(nonkw)}')
    if listing:
        print(f'Arguments: {args}')
    if isin is not None:
        if args[isin] in nonkw:
            print(f'{args[isin]} found in argument list.')
        else:
            print(f'Could not find {args[isin]}.')
"""

run(content, ['python test.py -h', 'python test.py -c -l -i txt arg1 arg2 txt',
              'python test.py -c -l -i txt arg1 arg2', 'python test.py -i'])

---

## II. `argparse`

Python argument parser library enables to create complex command line interfaces.
### 1. Basic workflow
- Initialize an argument parser
```python
parser = argparse.ArgumentParser()
```
- Add arguments
```python
parser.add_argument('--foo', help='foo help')
```
- Parse the values
```python
args = parser.parse_args()
```
- Use values
```python
print(args.foo)
```

In [ ]:
content = """
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--foo', help='foo help')
args = parser.parse_args()
print(args.foo)
"""
    
commands = ['python test.py -h', 'python test.py --foo arg1',
            'python test.py arg1', 'python test.py --bar']
run(content, commands)

### 2. Argument parameters

- positional | optional
    - positional
    ```python
    parser.add_argument('bar')
    ```
    - optional
    ```python
    parser.add_argument('--foo')
    ```
- argument data type
    - int, float, string
    ```python
    parser.add_argument('--foo', type='int')
    ```
    - boolean
    ```python
    parser.add_argument('--foo', action='store_true')
    ```
- requirement
    ```python
    parser.add_argument('--foo', required=True)
    ```
- default value
    ```python
    parser.add_argument('--foo', default='foo')
    ```
- argument cardinality
    - zero or one
    ```python
    parser.add_argument('--foo', nargs='?')
    ```
    - zero or more
    ```python
    parser.add_argument('--foo', nargs='*')
    ```
    - one or more
    ```python
    parser.add_argument('--foo', nargs='+')
    ```
- fixed set of values 
```python
parser.add_argument('--foo', choices=['foo', 'bar', 'baz'])
```
- specified variable name
```python
parser.add_argument('--foo', dest='bar')
```


In [ ]:
content = """
import argparse
parser = argparse.ArgumentParser(description="test script for argument parsing")
parser.add_argument('--foo', type=float, help='foo help')
args = parser.parse_args()
print(args.foo)
"""

with open('test.py', 'w') as f:
    f.write(content)
    
commands = ['python test.py -h', 'python test.py --foo arg1',
            'python test.py --foo 1.2', 'python test.py --bar']
run(content, commands)

---

## III. Exercises

### 1. Write a python calculator which gets the parameters through its CLI. For example:
```bash
python calc.py + 4 5
>>> 9
python calc.py - 4 5 3
>>> -4
python calc.py * 4 5 2 7
>>> 280
python calc.py / 4 2
>>> 2
```

### 2. Write a python script for file name generation:
- optional log level with fixed choices
- required date
- optional name with default 'user' value
- lowercase flag
- separator character (default: '_')
- possible extension
- any extra argument to append to filename

Example outputs:
```bash
python generate.py --log-level DEBUG --date 2018-11-08 --user admin --lowercase --separator - --extension log test
>>> debug-2018-11-08-admin-test.log 
python generate.py --date 2018-11-08
>>> 2018-11-08_user 
```